# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [17]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from g_key import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [18]:
file_path = "../output_data/cities.csv"
weather_df=pd.read_csv(file_path)
weather_df.drop_duplicates(subset=['City'])
weather_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
...,...,...,...,...,...,...,...,...,...,...
543,543,yabrud,0,SY,1558378840,32,33.97,36.66,80.60,8.05
544,544,paraiso,5,MX,1558378898,4,24.01,-104.61,84.20,16.11
545,545,veraval,0,FR,1558378898,62,49.65,0.71,61.00,8.05
546,546,novyy urgal,100,RU,1558378899,93,51.07,132.56,49.74,2.68


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [19]:
# Configure gmaps
gmaps.configure(api_key=gkey)

# Store latitude and longitude in locations
latlong = weather_df[["Lat", "Lng"]]

# Fill NaN values and convert to float
humidity = weather_df["Humidity"].astype(float)


In [20]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(latlong, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [21]:
conditions_df=weather_df.loc[(weather_df['Humidity']>=50) &(weather_df['Wind Speed']>=5) & (weather_df['Max Temp']<=65)]
conditions_df.dropna(how='all')
conditions_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
6,6,flinders,0,AU,1558378755,62,-34.58,150.85,60.01,11.41
8,8,tiksi,95,RU,1558378756,88,71.64,128.87,14.10,15.70
11,11,castro,0,CL,1558378756,70,-42.48,-73.76,48.20,9.17
...,...,...,...,...,...,...,...,...,...,...
532,532,burns lake,66,CA,1558378895,62,54.23,-125.76,56.40,5.21
535,535,villaviciosa,20,ES,1558378896,71,43.48,-5.43,59.00,6.93
540,540,talmenka,40,RU,1558378897,76,53.82,83.56,51.80,11.18
541,541,tamworth,64,AU,1558378897,87,-31.09,150.93,44.60,5.82


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [22]:
#begin the hotel df with city, country, lat, long
hotel_df=conditions_df.loc[:,['City','Country','Lat','Lng']]

#add the 'Hotel Name' column
hotel_df['Hotel Name']=''

#show dataframe
hotel_df


,City,Country,Lat,Lng,Hotel Name
0,ostrovnoy,RU,68.05,39.51,
4,cape town,ZA,-33.93,18.42,
6,flinders,AU,-34.58,150.85,
8,tiksi,RU,71.64,128.87,
11,castro,CL,-42.48,-73.76,
...,...,...,...,...,...
532,burns lake,CA,54.23,-125.76,
535,villaviciosa,ES,43.48,-5.43,
540,talmenka,RU,53.82,83.56,
541,tamworth,AU,-31.09,150.93,


In [23]:
#base url for search
base_url='https://maps.googleapis.com/maps/api/place/nearbysearch/json'

#parameters
params={'type':'hotel',
        'keyword':'hotel',
        'radius':5000,
         'key':'gkey'}


In [ ]:
#start for loop to 
for index, row in hotel_df.iterrows():
    lat=row['Lat']
    lng=row['Lng']
    name=row['City']
    
    
    params['Location']=(f'{lat},{lng}')
    
    #json request
    hotel_response=requests.get(base_url,params=params).json()
    
    # get search results
    query_response=hotel_response['results']

    #try excempt fo append search results to list
    try:
        hotel_df.loc[index,'Hotel Name']=query_response[0]['name']
    except:
        hotels.append('No hotel found')
    
#print the data frame
hotel_df

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
